# Class03: Data Exploration, Preprocessing, and Visualization

**Week04, Data Preparation**

ISM6136

&copy; 2023 Dr. Tim Smith


<a target="_blank" href="https://colab.research.google.com/github/prof-tcsmith/dm-f23/blob/main/W04/W04-Data-Exploration-and-Proprocessing.ipynb#offline=1">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

Introduction:
    
The dataset we are using in this notebook is a set of data on house sales in West Roxbuy, MA.

We are using this dataset to demonstrate the following common tasks:
1. Loading data into a dataframe
2. Explore the number of rows and columns found in the data
3. Rename column names
4. Drop any columns we are not interested in
5. Identify missing data
6. Drop rows with too many missing data measures
7. Impute the missing values for rows that are only missing very small number of values
8. Normalize (or Standardize) values in the dataframe



## 0. Import required packages

We use the pandas, the Python data anlysis library, for handling data. The API of this library is very similar to R data frames. See https://pandas.pydata.org/ for details.

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

import matplotlib.pylab as plt

## 1. Load data


The data is given in a CSV file, let's try loading the data into a dataframe

In [2]:
housing_df = pd.read_csv('WestRoxbury.csv') 

> Note that sometimes you may need to look at other read_csv parameters to deal with data that has anomolies, such as commas seperating thousands. See the documentation for read_csv found [here](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

Check if data is loaded into the dataframe by looking at the first 10 rows...

In [3]:
housing_df.head(10)

,TOTAL VALUE,TAX,LOT SQFT,YR BUILT,GROSS AREA,LIVING AREA,FLOORS,ROOMS,BEDROOMS,FULL BATH,HALF BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,6272,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,4170,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None
5,337.4,4244,5142,1950,2124,1060,1.0,6,3,1,0,1,1,Old
6,359.4,4521,5000,1954,3220,1916,2.0,7,3,1,1,1,0,None
7,320.4,4030,10000,1950,2208,1200,1.0,6,3,1,0,1,0,None
8,333.5,4195,6835,1958,2582,1092,1.0,5,3,1,0,1,1,Recent
9,409.4,5150,5093,1900,4818,2992,2.0,8,4,2,0,1,0,None


## 2. Explore number of rows and columns 

Let's look at the shape of the data frame. 

In [4]:
housing_df.shape

(5802, 14)

As we can see, the data consists of 5802 rows (observations) and 14 columns (variables). If we wanted to store these values into variables, we could do this as follows:

In [5]:
rows = housing_df.shape[0]
cols = housing_df.shape[1]
print(f"Rows={rows} and Cols={cols}")

Rows=5802 and Cols=14


## 3. Rename Columns

Often times the column names can have blank spaces, or other issues. We often change column names because of this.

We can get the columns names by displaying the columns attribute of the dataframe...

In [6]:
housing_df.columns

Index(['TOTAL VALUE ', 'TAX', 'LOT SQFT ', 'YR BUILT', 'GROSS AREA ',
       'LIVING AREA', 'FLOORS ', 'ROOMS', 'BEDROOMS ', 'FULL BATH',
       'HALF BATH', 'KITCHEN', 'FIREPLACE', 'REMODEL'],
      dtype='object')

Note that some of the column titles end with spaces and some consist of two space separated words. For further analysis it's more convenient to have column names which are single words and with no extra trailing or leading blank spaces.

If you want, you can rename one column name at a time using the rename command.

In the rename command you can specify individual columns by name and provide their new name using a dictionary. 

> Note that we use the `inplace` argument here. This means that the data frame is modified directly. By default, the modification is done on a copy and the copy returned by the method.

In [7]:
housing_df = housing_df.rename(columns={'TOTAL VALUE ': 'TOTAL_VALUE'})
housing_df.columns

Index(['TOTAL_VALUE', 'TAX', 'LOT SQFT ', 'YR BUILT', 'GROSS AREA ',
       'LIVING AREA', 'FLOORS ', 'ROOMS', 'BEDROOMS ', 'FULL BATH',
       'HALF BATH', 'KITCHEN', 'FIREPLACE', 'REMODEL'],
      dtype='object')

But, often we may want to process all the column names at once based on a pattern. The problem we have with the current columns names are the spaces. Let's change all the spaces to underscores using a list comprehension (for more on list comprehensions, see DataCamp Python DataScience Toolbox Part 2)

Below, instead of using the `rename` method, we create a modified copy of `columns` and assign to the `columns` field of the dataframe. We use a list comprehension

In [8]:
housing_df.columns = [s.strip().replace(' ', '_') for s in housing_df.columns] # note that we can change column names be simply assigning a new list of column names
housing_df.columns

Index(['TOTAL_VALUE', 'TAX', 'LOT_SQFT', 'YR_BUILT', 'GROSS_AREA',
       'LIVING_AREA', 'FLOORS', 'ROOMS', 'BEDROOMS', 'FULL_BATH', 'HALF_BATH',
       'KITCHEN', 'FIREPLACE', 'REMODEL'],
      dtype='object')

> NOTE: ```s.strip()``` returns a string with any trailing or leading whitespace removes. ```s.strip().replace(' ', '_')``` would take this returned string and replace at remaining blank spaces (the ones between words would be the only ones left) and replaces them with an underscore.

## 4. Drop any columns I'm not interested in

In this dataset, we've decided to remove the tax variable. To remove an columns use drop.


In [9]:
housing_df.drop(columns=['TAX']) # note: we can drop multiple columns by including more column names in the list

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,404.8,6762,1938,2594,1714,2.0,9,3,2,1,1,1,Recent
5798,407.9,9408,1950,2414,1333,2.0,6,3,1,1,1,1,None
5799,406.5,7198,1987,2480,1674,2.0,7,3,1,1,1,1,None
5800,308.7,6890,1946,2000,1000,1.0,5,2,1,0,1,0,None


An important thing to notice is that housing_df has not been altered by the ab ove drop command. The dop method returns a new dataframe with the column dropped, and this new dataframe is displayed but not stored over the original. So, if we look at the dataframe, we see that the TAX column still exists...

In [10]:
housing_df

,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,6272,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,4170,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,404.8,5092,6762,1938,2594,1714,2.0,9,3,2,1,1,1,Recent
5798,407.9,5131,9408,1950,2414,1333,2.0,6,3,1,1,1,1,None
5799,406.5,5113,7198,1987,2480,1674,2.0,7,3,1,1,1,1,None
5800,308.7,3883,6890,1946,2000,1000,1.0,5,2,1,0,1,0,None


> **When altering pandas dataframes, you often need to 'save' the results over the original. So, in order for us to drop the TAX column from the housing_df, we need to do the following...**

In [11]:
housing_df = housing_df.drop(columns=['TAX'])

In [12]:
housing_df

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,404.8,6762,1938,2594,1714,2.0,9,3,2,1,1,1,Recent
5798,407.9,9408,1950,2414,1333,2.0,6,3,1,1,1,1,None
5799,406.5,7198,1987,2480,1674,2.0,7,3,1,1,1,1,None
5800,308.7,6890,1946,2000,1000,1.0,5,2,1,0,1,0,None


## 5. Identify any categorical data in the dataframne, and be sure that the data is loaded correctly

Categorical data is data that represents categories. Categorical data shouldn't be treated like a number, as it isn't one 

Notice in our data we have the variable (column) REMODEL, and this variable contains one of three possible strings None, Recent, or Old.

First' let's look at this REMODEL variable...

In [13]:
housing_df['REMODEL'].unique()

array(['None', 'Recent', 'Old'], dtype=object)

Let's see what data types pandas has chosen for our variables (columns)?

In [14]:
housing_df.dtypes

TOTAL_VALUE    float64
LOT_SQFT         int64
YR_BUILT         int64
GROSS_AREA       int64
LIVING_AREA      int64
FLOORS         float64
ROOMS            int64
BEDROOMS         int64
FULL_BATH        int64
HALF_BATH        int64
KITCHEN          int64
FIREPLACE        int64
REMODEL         object
dtype: object

The REMODEL column is set as an 'object'. This is a general type and we want this to be translated into a categorical type, therefore need to change the comumn REMODEL from object to categorical.

In [15]:
print(housing_df.REMODEL.dtype) # double check and confirm that the column is an 'object' type
housing_df.REMODEL = housing_df.REMODEL.astype('category') # change the column type to a categorical variable
print(housing_df.REMODEL.cat.categories)  # Print the categories found
print(housing_df.REMODEL.dtype)  # now, let's confirm that the column type has been changed to categorical

object
Index(['None', 'Old', 'Recent'], dtype='object')
category


## 6. Identify and Handle Any Missing Data

First, let's identify any columns that contain missing data...

In [16]:
housing_df.isna().sum()

TOTAL_VALUE    0
LOT_SQFT       0
YR_BUILT       0
GROSS_AREA     0
LIVING_AREA    0
FLOORS         0
ROOMS          0
BEDROOMS       0
FULL_BATH      0
HALF_BATH      0
KITCHEN        0
FIREPLACE      0
REMODEL        0
dtype: int64

There doesn't seem to be any missing values in this data. For demonstrations purpuses, let's randomly select 10 values for BEDROOMS and make them nan (which means, not a number, but this is used by pandas to indicate a missing value).

In [17]:
print(f"Number of rows with valid BEDROOMS values before: {housing_df['BEDROOMS'].count()}") 
add_missing_rows = housing_df.sample(10).index # create a random selection of rows that we will use as to overwrite with a NAN for BEDROOMS
housing_df.loc[add_missing_rows, 'BEDROOMS'] = np.nan  # change these rows to have BEDROOM values NAN
print(f"Number of rows with valid BEDROOMS values after setting to NAN: {housing_df['BEDROOMS'].count()}") 


Number of rows with valid BEDROOMS values before: 5802
Number of rows with valid BEDROOMS values after setting to NAN: 5792


There are many ways we could identify any missing values (nan). If we want to check all variables to find if any have missing values, we can do the following.

In [18]:
housing_df.isnull().sum()

TOTAL_VALUE     0
LOT_SQFT        0
YR_BUILT        0
GROSS_AREA      0
LIVING_AREA     0
FLOORS          0
ROOMS           0
BEDROOMS       10
FULL_BATH       0
HALF_BATH       0
KITCHEN         0
FIREPLACE       0
REMODEL         0
dtype: int64

As we expected, we find that BEDROOMS has 10 missing values

In [19]:
housing_df['BEDROOMS'].count() # note that we can count the number of bedroom values, and find that it's less than the number of rows in the dataframe, therefore must now contain missing values

5792

When we find NAN's (Missing numbers/values), we have two general approaches we can take. 

1. Replace the NAN with the mean of the other values in the column (this is often referred to as **'imputing'** a value).
2. Drop the row (observation) that contains the NAN

### Imputing missing values

Often, we may have a row of values but only a small portion have a missing value. For instance, if a row has 10 values and only one of them is empty, it would be advantageous to keep the other 9 values and 'impute' the missing value. Imputing a value typically is done by replaing the missing value with the mean (average) of all values for the variable.

In the code below, I replace the missing values using the median of the remaining values.

NOTE: By default, the `median` method of a pandas dataframe ignores NA values. This is in contrast to R where this must be specified explicitly.

In [20]:
medianBedrooms = housing_df['BEDROOMS'].median()
housing_df.BEDROOMS = housing_df.BEDROOMS.fillna(value=medianBedrooms)
print(f"Number of rows with valid BEDROOMS values after filling NA values: {housing_df['BEDROOMS'].count()}")

Number of rows with valid BEDROOMS values after filling NA values: 5802


### Droping rows that have over a certain number of missing values

For demonstration purposes, let's add more NAN's into our data. This time we will pick a sample of rows and change 3 of the values to NAN.

In [21]:
add_missing_values = housing_df.iloc[0:5,:].index # let's overwrite the first 10 rows with NAN's for bedrooms, fireplace and rooms
housing_df.loc[add_missing_values, 'BEDROOMS'] = np.nan  # change these rows to have BEDROOM values NAN
housing_df.loc[add_missing_values, 'FIREPLACE'] = np.nan  # change these rows to have BEDROOM values NAN
housing_df.loc[add_missing_values, 'ROOMS'] = np.nan  # change these rows to have BEDROOM values NAN

In [22]:
housing_df.head(10)

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,9965,1880,2436,1352,2.0,NaN,NaN,1,1,1,NaN,None
1,412.6,6590,1945,3108,1976,2.0,NaN,NaN,2,1,1,NaN,Recent
2,330.1,7500,1890,2294,1371,2.0,NaN,NaN,1,1,1,NaN,None
3,498.6,13773,1957,5032,2608,1.0,NaN,NaN,1,1,1,NaN,None
4,331.5,5000,1910,2370,1438,2.0,NaN,NaN,2,0,1,NaN,None
5,337.4,5142,1950,2124,1060,1.0,6.0,3.0,1,0,1,1.0,Old
6,359.4,5000,1954,3220,1916,2.0,7.0,3.0,1,1,1,0.0,None
7,320.4,10000,1950,2208,1200,1.0,6.0,3.0,1,0,1,0.0,None
8,333.5,6835,1958,2582,1092,1.0,5.0,3.0,1,0,1,1.0,Recent
9,409.4,5093,1900,4818,2992,2.0,8.0,4.0,2,0,1,0.0,None


Now, let's remove any rows that have more than one missing value. 

In [23]:
housing_df = housing_df[housing_df.isnull().sum(axis=1) < 3]
housing_df

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
5,337.4,5142,1950,2124,1060,1.0,6.0,3.0,1,0,1,1.0,Old
6,359.4,5000,1954,3220,1916,2.0,7.0,3.0,1,1,1,0.0,None
7,320.4,10000,1950,2208,1200,1.0,6.0,3.0,1,0,1,0.0,None
8,333.5,6835,1958,2582,1092,1.0,5.0,3.0,1,0,1,1.0,Recent
9,409.4,5093,1900,4818,2992,2.0,8.0,4.0,2,0,1,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,404.8,6762,1938,2594,1714,2.0,9.0,3.0,2,1,1,1.0,Recent
5798,407.9,9408,1950,2414,1333,2.0,6.0,3.0,1,1,1,1.0,None
5799,406.5,7198,1987,2480,1674,2.0,7.0,3.0,1,1,1,1.0,None
5800,308.7,6890,1946,2000,1000,1.0,5.0,2.0,1,0,1,0.0,None


## 7. Converting categorical data into dummy variables 

Pandas provides a method to convert factors into dummy variables. 

If you're not familiar with what a dummy variable is, notice that in our data the REMODEL variable is categorical and has three categories. These categories shouldn't be intgerpreted as ordered, or equal distance apart. When we find such variables, we need to encode them using either a dummy variable method, or 'one hot encoding'. This is a topic we will discuss more later. For now, let's look how we can encode the REMODEL variable into dummy variables....

In [24]:
housing_df = pd.get_dummies(housing_df, prefix_sep='_', drop_first=True)
housing_df.columns

Index(['TOTAL_VALUE', 'LOT_SQFT', 'YR_BUILT', 'GROSS_AREA', 'LIVING_AREA',
       'FLOORS', 'ROOMS', 'BEDROOMS', 'FULL_BATH', 'HALF_BATH', 'KITCHEN',
       'FIREPLACE', 'REMODEL_Old', 'REMODEL_Recent'],
      dtype='object')

Note that we had three possible categories for REMODEL: Old, Recent or None. 

Using the dummy variable approach, we now represent this data using two 'dummy variables'. 

If REMODEL is Old, the REMODEL_Old will be 1 and REMODEL_Recent will be 0. 

Otherwise, if the REMODEL is recent, then REMODEL_RECENT will be 1 and REMODEL_old will be 0. 

Finally, if REMODEL is None, they will both be zero. 


In [25]:
print(housing_df.loc[:, 'REMODEL_Old':'REMODEL_Recent'].head(5))

   REMODEL_Old  REMODEL_Recent
5            1               0
6            0               0
7            0               0
8            0               1
9            0               0


## 8. Table - scaling data (standardizing)

We often rescale data within a dataframe. Rescaling is often necessary to remove the influence of difference measurement scales. Some variables can have very large values, and others have rather small. This is a difference in scale, and some machine learning algorithms work best if we 'normalize' the data so that scale is not a factore.

In [26]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
df = housing_df.copy()

# More common scaling technique is to normalize the data (each variable value is scaled using the variables standard deviation)

# you could use pandas as follows...
norm_df = (housing_df - housing_df.mean()) / housing_df.std()

# or, you could use scikit-learn's StandardScaler...: 
scaler = StandardScaler()
norm_df = pd.DataFrame(scaler.fit_transform(housing_df), 
                       index=housing_df.index, columns=housing_df.columns)

# Other rescaling techniques - min_max scaling

# You can implement mix_max scaling using pandas:
rescaled_df = (housing_df - housing_df.min()) / (housing_df.max() - housing_df.min())

# Or, you could use scikit-learn's MinMaxScaler:
scaler = MinMaxScaler()
rescaled_df = pd.DataFrame(scaler.fit_transform(housing_df), 
                       index=housing_df.index, columns=housing_df.columns)

The standardization of the dataset may give a <code>DataConversionWarning</code>. This informs you that the integer columns in the dataframe are automatically converted to real numbers (<code>float64</code>). This is expected and you can therefore ignore this warning. If you want to suppress the warning, you can explicitly convert the integer columns to real numbers</p>
<pre>
# Option 1: Identify all integer columns, remove personal loan, 
# and change their type
intColumns = [c for c in housing_df.columns if housing_df[c].dtype == 'int']
housing_df[intColumns] = housing_df[intColumns].astype('float64')
</pre>
Alternatively, you can suppress the warning as follows:
<pre>
# Option 2: use the warnings package to suppress the display of the warning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    norm_df = pd.DataFrame(scaler.fit_transform(housing_df), 
                       index=housing_df.index, columns=housing_df.columns)    
</pre>

## 9. Practice Accessing Data in a Dataframe

Pandas uses two methods to access rows in a data frame; `loc` and `iloc`. The `loc` method is more general and allows accessing rows using labels. The `iloc` method on the other hand only allows using integer numbers. To specify a range of rows use the slice notation, e.g. `0:9`.

<div class='alert alert-info'>Note that in contrast to R, Python uses 0-indexing, which means that indices start at 0 and not at 1.</div>

To show the first four rows of the data frame, you can use the following commands.

In [27]:
housing_df.loc[0:3]  # for loc, the second index in the slice is inclusive

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent


In [28]:
housing_df.iloc[0:4]  # for loc, the second index in the slice is exclusive

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
5,337.4,5142,1950,2124,1060,1.0,6.0,3.0,1,0,1,1.0,1,0
6,359.4,5000,1954,3220,1916,2.0,7.0,3.0,1,1,1,0.0,0,0
7,320.4,10000,1950,2208,1200,1.0,6.0,3.0,1,0,1,0.0,0,0
8,333.5,6835,1958,2582,1092,1.0,5.0,3.0,1,0,1,1.0,0,1


Note the difference in the two methods with respect to the slice notation! For consistency with how slices are defined in Python, we will use the `iloc` method mostly from here on.

Next, show the first ten rows of the first column

In [29]:
housing_df['TOTAL_VALUE'].iloc[0:10]
housing_df.iloc[0:10]['TOTAL_VALUE']  # the order is not important
housing_df.iloc[0:10].TOTAL_VALUE

5     337.4
6     359.4
7     320.4
8     333.5
9     409.4
10    313.0
11    344.5
12    315.5
13    575.0
14    326.2
Name: TOTAL_VALUE, dtype: float64

Show the fifth row of the first 10 columns. The `iloc` methods allows specifying the rows and columns within one set of brackets. `dataframe.iloc[rows, columns]`

In [30]:
housing_df.iloc[4][0:10]
housing_df.iloc[4, 0:10]  # this is equivalent

TOTAL_VALUE     409.4
LOT_SQFT       5093.0
YR_BUILT       1900.0
GROSS_AREA     4818.0
LIVING_AREA    2992.0
FLOORS            2.0
ROOMS             8.0
BEDROOMS          4.0
FULL_BATH         2.0
HALF_BATH         0.0
Name: 9, dtype: float64

If you prefer to preserve the data frame format, use a slice for the rows as well.

In [31]:
housing_df.iloc[4:5, 0:10]

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH
9,409.4,5093,1900,4818,2992,2.0,8.0,4.0,2,0


Use the `pd.concat` method if you want to combine non-consecutive columns into a new data frame. The `axis` argument specifies the dimension along which the concatenation happens, 0=rows, 1=columns.

In [32]:
pd.concat([housing_df.iloc[4:6,0:2], housing_df.iloc[4:6,4:6]], axis=1)

,TOTAL_VALUE,LOT_SQFT,LIVING_AREA,FLOORS
9,409.4,5093,2992,2.0
10,313.0,5000,1485,1.5


To specify a full column, use the `:` on its own.
```
housing.iloc[:,0:1]
```
A often more practical way is to use the column name as follows

In [33]:
housing_df['TOTAL_VALUE']

5       337.4
6       359.4
7       320.4
8       333.5
9       409.4
        ...  
5797    404.8
5798    407.9
5799    406.5
5800    308.7
5801    447.6
Name: TOTAL_VALUE, Length: 5797, dtype: float64

To shorten the number of values we see (the above list is long) we can subset the column using a slice.

In [34]:
housing_df['TOTAL_VALUE'][0:10]

5     337.4
6     359.4
7     320.4
8     333.5
9     409.4
10    313.0
11    344.5
12    315.5
13    575.0
14    326.2
Name: TOTAL_VALUE, dtype: float64

Pandas provides a number of ways to access statistics of the columns.

In [35]:
print('Number of rows ', len(housing_df['TOTAL_VALUE']))
print('Mean of TOTAL_VALUE ', housing_df['TOTAL_VALUE'].mean())

Number of rows  5797
Mean of TOTAL_VALUE  392.6937239951709


A data frame also has the method `describe` that prints a number of common statistics 

In [36]:
housing_df['TOTAL_VALUE'].describe()

count    5797.000000
mean      392.693724
std        99.201379
min       105.000000
25%       325.000000
50%       375.900000
75%       438.800000
max      1217.800000
Name: TOTAL_VALUE, dtype: float64

In [37]:
housing_df.describe()

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
count,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000,5797.000000
mean,392.693724,6276.110747,1936.762463,2924.735898,1656.986027,1.683629,6.993962,3.229429,1.296705,0.613766,1.015353,0.740383,0.100224,0.150768
std,99.201379,2668.499941,35.989613,883.837223,540.493683,0.444908,1.437374,0.846466,0.522058,0.533912,0.122962,0.565007,0.300325,0.357853
min,105.000000,997.000000,0.000000,821.000000,504.000000,1.000000,3.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,325.000000,4770.000000,1920.000000,2347.000000,1308.000000,1.000000,6.000000,3.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,375.900000,5682.000000,1935.000000,2700.000000,1549.000000,2.000000,7.000000,3.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,438.800000,7020.000000,1955.000000,3239.000000,1873.000000,2.000000,8.000000,4.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1217.800000,46411.000000,2011.000000,8154.000000,5289.000000,3.000000,14.000000,9.000000,5.000000,3.000000,2.000000,4.000000,1.000000,1.000000


## 10 Subsampling data (or randoming drawing data from a dataframe)

Use the `sample` method to retrieve a random sample of observations. Here we sample 5 observations without replacement.

In [38]:
housing_df.sample(5)

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
2299,385.7,7185,1920,2493,1426,2.0,7.0,3.0,1,0,1,1.0,0,0
1721,329.9,3160,1970,2604,1536,2.0,7.0,4.0,1,1,1,1.0,0,0
5465,359.0,3582,1956,2484,1765,1.0,7.0,3.0,1,1,1,1.0,0,0
4040,387.5,6500,1920,2512,1452,2.0,7.0,4.0,1,1,1,1.0,0,0
440,286.5,4272,1965,3084,1512,1.0,8.0,3.0,1,1,1,1.0,0,0


The sample method allows to specify weights for the individual rows. We use this here to oversample houses with over 10 rooms.

In [39]:
weights = [0.9 if rooms > 10 else 0.01 for rooms in housing_df.ROOMS] # another list comprehension, see datacamp "toolbox part 2" course.
housing_df.sample(5, weights=weights)

,TOTAL_VALUE,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL_Old,REMODEL_Recent
965,479.2,7162,1900,4941,2564,2.0,9.0,4.0,1,1,1,1.0,1,0
2911,599.7,8040,1886,5406,3112,2.0,12.0,4.0,1,1,1,1.0,0,0
1534,282.8,3750,1945,2286,1254,1.5,6.0,3.0,2,0,1,1.0,0,0
3498,328.9,4000,1934,2122,1318,2.0,6.0,3.0,1,1,1,1.0,0,0
4657,791.6,7629,1890,6321,3521,2.0,12.0,6.0,4,1,1,2.0,0,1


## 11. Partitioning data into training and validation

Split the dataset into training (70%) and validation (30%) sets. 

Randomly sample 70% of the dataset into a new data frame `train_data`. The remaining 40% serve as validation.

In [40]:
# random_state is set to a defined value to get the same partitions when re-running the code
train_data= housing_df.sample(frac=0.7, random_state=1)
# assign rows that are not already in the training set, into validation 
valid_data = housing_df.drop(train_data.index)

print('Training   : ', train_data.shape)
print('Validation : ', valid_data.shape)
print()

# alternative way using scikit-learn
train_data, valid_data = train_test_split(housing_df, test_size=0.40, random_state=1)
print('Training   : ', train_data.shape)
print('Validation : ', valid_data.shape)

Training   :  (3478, 14)
Validation :  (2319, 14)

Training   :  (3478, 14)
Validation :  (2319, 14)


Partition the dataset into training (50%), validation (30%), and test sets (20%). 

In [41]:
# randomly sample 60% of the row IDs for training
train_data = housing_df.sample(frac=0.5, random_state=1)
# sample 30% of the row IDs into the validation set, drawing only from records
# not already in the training set; 60% of 50% is 30%
valid_data = housing_df.drop(train_data.index).sample(frac=0.6, random_state=1)  
# the remaining 20% rows serve as test
test_data = housing_df.drop(train_data.index).drop(valid_data.index)

print('Training   : ', train_data.shape)
print('Validation : ', valid_data.shape)
print('Test       : ', test_data.shape)
print()

# alternative way using scikit-learn
train_data, temp = train_test_split(housing_df, test_size=0.5, random_state=1)
valid_data, test_data = train_test_split(temp, test_size=0.4, random_state=1)
print('Training   : ', train_data.shape)
print('Validation : ', valid_data.shape)
print('Test       : ', test_data.shape)

Training   :  (2898, 14)
Validation :  (1739, 14)
Test       :  (1160, 14)

Training   :  (2898, 14)
Validation :  (1739, 14)
Test       :  (1160, 14)
